In [ ]:
import numpy as np
import pysindy as ps
from pysindy.feature_library import PolynomialLibrary, FourierLibrary
import matplotlib.pyplot as plt

from generate_data import generate_tracking_data, generate_discrete_tracking_data

In [ ]:
T = 1e-1
t = np.linspace(0, 20, 201)
x0 = [-0.1, 0.2, -0.1]
x0_val = [0.1, 0.1, 0]

x, x_dot = generate_tracking_data(t=t, x0=x0)
x_val, _ = generate_tracking_data(t=t, x0=x0_val)

xk = generate_discrete_tracking_data(t=t, x0=x0, T=T)
xk_val = generate_discrete_tracking_data(t=t, x0=x0_val, T=T)

In [ ]:
pr = int(3 - np.log10(T))
model = ps.SINDy(
    feature_library=PolynomialLibrary(degree=3),
    # feature_library=FourierLibrary(n_frequencies=1),
    optimizer=ps.STLSQ(threshold=0.0001),
    feature_names=[f'x{i+1}' for i in range(len(x[0]))],
    discrete_time=True
    )
model.fit(x=xk)
# model.fit(x=x, t=t, x_dot=x_dot)
model.print(precision=pr)

In [ ]:
x_sim = model.simulate(x0=x0_val, t=len(t))
mse = ((x_sim - xk_val)**2).mean(axis=0)
print(f'Błąd średniokwadratowy x1: {mse[0]}, x2: {mse[1]}, x3: {mse[2]}')

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(10, 8), sharex=True)

labels = [
    ("Kąt natarcia", "Model kąt natarcia"),
    ("Kąt nachylenia", "Model kąta nachylenia"),
    ("Współczynnik nachylenia", "Model współczynnika nachylenia")]

for i in range(3):
    axs[i].plot(t, xk_val[:, i], label=labels[i][0], color=f"C{2*i}")
    axs[i].plot(t, x_sim[:, i], '--', label=labels[i][1], color=f"C{2*i+1}")
    axs[i].legend(fontsize=14)
    axs[i].set_ylabel("Rad", fontsize=16)
    axs[i].grid()
    axs[i].set_xlim(0, 20)
    axs[i].tick_params(axis='both', labelsize=14)

axs[2].set_xlabel("Czas [s]", fontsize=16)
plt.tight_layout()
plt.show()